In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
pip install catboost

In [ ]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import seaborn as sns
import plotly.express as px
from catboost import CatBoostRegressor
from ipywidgets import widgets
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from plotly.subplots import make_subplots
import datetime as dt
pd.options.display.max_colwidth = 150

# Шаг 1. Загрузим данные и посмотрим датасет


In [ ]:
# Загрузим дата сет и посмотрим первые три строки
air = pd.read_csv('/kaggle/input/ab-nyc-2019/AB_NYC_2019.csv')
air.head(3)

In [ ]:
# Проверим пустые значения в файлах
def missing_values(df):
    total = df.isna().sum()
    percentage = round(total/df.shape[0]*100,2)
    return pd.concat([total,percentage],axis = 1, keys = ['total','percentage'])

# Проверим дублирующиеся значения в файлах
def duplicates_values(df):
    dup = []
    columns = df.columns
    for row in df.columns:
        dup.append(df.duplicated().sum())
    return pd.concat([pd.Series(columns),pd.Series(dup)],axis = 1, keys = ['columns','duplicate count'])

# Посмотрим общую информацию о файлах
def df_info(df):
    df.info()
    display(df.describe())
    display(df.head(5))
    display(df.shape)

In [ ]:
missing_values(air)

In [ ]:
duplicates_values(air)

In [ ]:
df_info(air)

****
## Вывод:

*   данные загружены(размер 48895 строк и 16 колонок)
*   в датасете есть пустые значения в столбцах (name,last_review,reviews_per_month,host_name)
*   дублирующихся значений нет
****




  




****
# Шаг 2. Сделаем предобработку данных

*   Заменим пустые значения в колонке name на unknown
*   Заменим пустые значения в колонке host_name на unknown
*   Колонки last_review и reviews_per_month удалим, так как если эти кваритры(комнаты никто не смотрит , то и смысла в них особого нет)
*   Заменим тип данных в колонке last_review на date.
*   Так же в датасете есть выбросы в колонках «price» и «minimum nights», 10000$ и 1250 nights



In [ ]:
# Заменим пустые значения в колонке name
air['name'] = air['name'].replace(np.NaN,'unknown')

In [ ]:
# Заменим пустые значения в колонке host_name
air['host_name'] = air['host_name'].replace(np.NaN,'unknown')

In [ ]:
df = air.dropna()

In [ ]:
df['price'].quantile(0.99)

In [ ]:
df['minimum_nights'].quantile(0.99)

In [ ]:
# Сделаем срез , что бы убрать выбросы (по колонкам price и minimum_nights)
df = df.query('price <= 671.16 and minimum_nights <= 31')

In [ ]:
# Поменяем тип данных в колонке last_review на date
df['last_review'] =df['last_review'].map(lambda x: dt.datetime.strptime(x, '%Y-%m-%d'))

In [ ]:
print('Количество потерянных данных =',\
      round(100-len(df)/len(air)*100, 2),'%')

***
### Вывод 

1.   Заменили пустые значения в колонках «host_name» и «name»
2.   Сделал срез по данным, взял значение из колонки «price» == 671.1$ так как 99% значений лежат в этом диапозоне. Взял значение из колоки «minimum_nights» == 31, так как 99% значений лежат именно в этом диапозоне
3.  В процессе предобработки мы потеряли 21.95 % от первоночального датасета
****



******

# Шаг 3. EDA анализ

1.   В датасете есть координаты всех объектов, я хочу получить дистанцию каждого объекта до центра, центрального парка и Эмпайр-стейт-билдинг, это понадобится для дальнейшего ML
2.   Визуализируем данные( построим гисторамму, sunburst и отрисуем карту со всеми объектами)



### 3.1 Получим дистанцию

In [ ]:
# сделаем кортеж из колонки долготы и широты
df['lat_long'] = df[['latitude','longitude']].apply(tuple, axis=1)

In [ ]:
# координаты Нью-Йорка
new_york = (40.714599, -74.002791)

In [ ]:
# координаты центрального парка
center_park = (40.7825, -73.965493)

In [ ]:
# координаты Эмпайр-стейт-билдинг
empire_state_building = (40.748391, -73.985543)

In [ ]:
# координаты Таймс сквер
times_square = (40.755878, -73.986729)

In [ ]:
# посчитаем дистанцию
from geopy import distance

distance_to_times_square = []
distance_to_empire_state_building = []
distance_to_center_park = []
distance_to_center = []


for i in df['lat_long']:
  distance_1 = (distance.distance(i, new_york).km)
  distance_2 = (distance.distance(i, center_park).km)
  distance_3 = (distance.distance(i, empire_state_building).km)
  distance_4 = (distance.distance(i, times_square).km)


  distance_to_center.append(distance_1)
  distance_to_center_park.append(distance_2)
  distance_to_empire_state_building.append(distance_3)
  distance_to_times_square.append(distance_4)


df['distance_to_center']=distance_to_center
df['distance_to_center_park']=distance_to_center_park
df['distance_to_empire_state_building']=distance_to_empire_state_building
df['distance_to_times_square']=distance_to_times_square

In [ ]:
df.sample(2)

***

### 3.2 Отрисуем гистограмму (по удалению обьектов от центра)

In [ ]:
df.room_type.unique()

In [ ]:
x = df.query('room_type == "Private room"')['distance_to_center']
y = df.query('room_type == "Entire home/apt"')['distance_to_center']
z = df.query('room_type == "Shared room"')['distance_to_center']

fig = go.Figure()

fig.add_trace(go.Histogram(x=x,name="Private room",marker_color='green', opacity=0.8))
fig.add_trace(go.Histogram(x=y,name="Entire home/apt",marker_color='blue',opacity=0.7))
fig.add_trace(go.Histogram(x=z,name="Shared room",marker_color='red'))

fig.update_layout(barmode='overlay')

fig.update_layout(
    title_text='Количество cдаваемых обьектов по удалению от центра',
    xaxis_title_text='distance to center',
    yaxis_title_text='count',
    bargap=0.2,
    bargroupgap=0.1
)

fig.show()

### Вывод:

1.   Для Private room - количество сдаваемых объектов возрастает до 5 км, потом начинает снижаться
2.   Для Entire home/apt - возрастает до 5-6 км, потом начинает постепено снижаться
3.   Для Shared room - самое большое количестов объектов находиться на расстоянии 5-6 км и 8-9 км



****
### 3.3 Отрисуем sunburst
****

In [ ]:
sunburst=df.groupby(['neighbourhood_group','neighbourhood']).agg(count = ('room_type','count'),
                                                         median_price = ('price','median'),
                                                         max_price = ('price','max'),
                                                         min_price = ('price','min')).reset_index()
sunburst.sample(3)                                                        

In [ ]:
fig = px.sunburst(sunburst, path=['neighbourhood_group', 'neighbourhood'],
                  values='median_price',color='median_price',color_continuous_scale='RdBu',)
fig.update_layout(
    margin = dict(t=0, l=0, r=100, b=10)
)
fig.show()

In [ ]:
df.groupby(['neighbourhood_group']).\
agg(count = ('room_type','count'),
    median_price = ('price','median'),
    max_price = ('price','max'),
    min_price = ('price','min')).reset_index()


*****
### Вывод

1.  Больше всего жилья сдается в районе Бруклина и Манхеттена, 16266 и 16174 сответственно
2.  Самая высокая цена по медиане в районе Манхеттен 140 $
3.  По макc.цене все районы примерно находятся в одной ценовой категории
4.  На графике sunburst можно посмотреть самые дорогие и районы  по мед.цене

*****


****
### 3.4 Отрисуем карту с местоположением всех объектов в Нью-Йорке
****

In [ ]:
fig = px.scatter_mapbox(df, lat='latitude', 
                        lon='longitude', 
                        hover_name='room_type',color = 'price',
                        zoom=9, height=800, width=1000,size_max=10, )
fig.update_layout(
    mapbox_style="open-street-map" )#"open-street-map", "carto-positron", "carto-darkmatter", "stamen-terrain", "stamen-toner" or "stamen-watercolor" 
fig.show()

### Вывод

*   Как можно увидеть на отрисованной,карте самое большое количество объектов с высокой ценой расположенно в районе Манхеттана




# ШАГ 4. Применим ML для прогноза цены ( буду использовать catboost)

In [ ]:
plt.figure(figsize=(10,10))
sns.set(style="whitegrid")
sns.heatmap(df.corr()[['price']], 
            annot = True,
            square=True,
            fmt='.2f',
            cmap= 'coolwarm',
            linewidths=1, 
            linecolor='black')
plt.show()

In [ ]:
df.columns

In [ ]:
features = [x for x in df.columns if x not in ['price']]
X = df[features]
y = df['price']

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, random_state=1)

numerical_cols = [cname for cname in X_train.columns if 
                X_train[cname].dtype in ['int64', 'float64']]

categorical_cols = [cname for cname in X_train.columns if
                    X_train[cname].nunique() <= 20  and 
                    X_train[cname].dtype == "object"]


numerical_transformer = SimpleImputer(strategy='mean')

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols )
])
 

CatBoost = CatBoostRegressor(verbose=5000,random_state=42,iterations=10000)

random_clf = Pipeline(steps=[('preprocessor', preprocessor),
                             ('CatBoost', CatBoost)])

random_clf.fit(X_train, y_train)


random_preds = random_clf.predict(X_valid)

In [ ]:
print('---------------------------------------')
print('MAE: {:.2f}'.format(mean_absolute_error(y_valid, random_preds)))
print('---------------------------------------')
print('MSE: {:.2f}'.format(mean_squared_error(y_valid, random_preds)))
print('---------------------------------------')
print('R2: {:.2f}'.format(r2_score(y_valid, random_preds)))
print('---------------------------------------')

In [ ]:
df['predict_price'] = random_clf.predict(X)

In [ ]:
df['error'] = np.abs(df['price'] - df['predict_price'])

In [ ]:
print('---------------------------------')
print('Средняя ошибка:',round(df['error'].mean(),2),"$")
print('---------------------------------')
print('Медиана ошибки:',round(df['error'].median(),2),"$")
print('---------------------------------')
print('Медиана ошибки для категории жилья Private room =', round(df.query('room_type == "Private room"')['error'].median(),2), "$")
print('---------------------------------')
print('Медиана ошибки для категории жилья Entire home/apt =', round(df.query('room_type == "Entire home/apt"')['error'].median(),2), "$")
print('---------------------------------')
print('Медиана ошибки для категории жилья Shared room =', round(df.query('room_type == "Shared room"')['error'].median(),2), "$")
print('---------------------------------')

# Вывод:

***Модель показывает следующие результаты***


MAE: 39.52
---------------------------------------
MSE: 3892.31
---------------------------------------
R2: 0.52
---------------------------------------

Средняя ошибка: 34.19 
---------------------------------
Медиана ошибки: 21.5 
---------------------------------
Медиана ошибки для категории жилья Private room = 14.0
---------------------------------
Медиана ошибки для категории жилья Entire home/apt = 34.96
---------------------------------
Медиана ошибки для категории жилья Shared room = 10.66
---------------------------------


